This notebook is used to test the `cartan_plugin.py` file and to make sure the plugin is installed correctly.

In [1]:
from qiskit_cartan.cartan_plugin import *

import logging
logging.basicConfig(level='DEBUG')

#### Check `synth_cartan` works as intended with an example Heisenberg Hamiltonian

In [2]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
test_heisenberg_ham = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], coeffs = [1,1,1,1,1,1])
test_time_evolve = 3
test_paulievolutiongate = PauliEvolutionGate(test_heisenberg_ham, time=test_time_evolve)
qc = synth_cartan(test_paulievolutiongate, 0)
print(qc)

Optimization terminated successfully.
         Current function value: -2.586419
         Iterations: 13
         Function evaluations: 22
         Gradient evaluations: 22
--- 0.06998658180236816 seconds ---
Optimization Error:
3.606967140699748e-12
     ┌────────────────────────┐┌────────────────────────┐»
q_0: ┤0                       ├┤0                       ├»
     │                        ││                        │»
q_1: ┤1 exp(-it XYZ)(-1.3459) ├┤1 exp(-it XZY)(0.77382) ├»
     │                        ││                        │»
q_2: ┤2                       ├┤2                       ├»
     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌───────────────────────┐»
«q_0: ┤0                       ├┤0                      ├»
«     │                        ││                       │»
«q_1: ┤1 exp(-it ZYX)(-1.1724) ├┤1 exp(-it YZX)(1.7887) ├»
«     │                        ││                       │»
«q_2: ┤2                       ├┤2       

In [3]:
from qiskit import Aer, transpile

unitary_simulator = Aer.get_backend('unitary_simulator')
qc_transp = transpile(qc, unitary_simulator)
final_unitary_cartan = unitary_simulator.run(qc_transp).result().get_unitary()

import scipy
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], 
                  np.array([1, 1, 1, 1, 1, 1])).to_matrix()
propagator = scipy.linalg.expm(-1j*H*test_time_evolve)
matrix_norm = np.linalg.norm(propagator - final_unitary_cartan)
print("\n")
print("matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: \n", matrix_norm)

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 1.00422 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.99683 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
DEBUG:stev



matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: 
 7.494421707149518e-07


#### Check our plugin is installed

In [4]:
from qiskit.transpiler.passes.synthesis.plugin import HighLevelSynthesisPluginManager
HLS_plugin_manager = HighLevelSynthesisPluginManager()
print(HLS_plugin_manager.plugins.names())

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

['clifford.default', 'linear_function.default', 'permutation.acg', 'permutation.basic', 'permutation.default', 'permutation.kms', 'PauliEvolution.cartan']


#### Test the pass

In [5]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
from qiskit.transpiler import PassManager

qc_before = QuantumCircuit(3)
qc_before.append(test_paulievolutiongate, range(3))
print("before: ")
qc_before.draw()

before: 


┌──────────────────────────────────────────────────┐
q_0: ┤0                                                 ├
     │                                                  │
q_1: ┤1 exp(-it (XXI + YYI + ZZI + IXX + IYY + IZZ))(3) ├
     │                                                  │
q_2: ┤2                                                 ├
     └──────────────────────────────────────────────────┘

In [12]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HLSConfig

hls_config = HLSConfig(PauliEvolution=[("cartan", {"random_seed":10})])
pm = PassManager()
pm.append(HighLevelSynthesis(hls_config=hls_config))
qc_after = pm.run(qc_before)
qc_after.draw()

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

Running Cartan Synthesis Plugin...
Optimization terminated successfully.
         Current function value: -2.586419
         Iterations: 19
         Function evaluations: 33
         Gradient evaluations: 33
--- 0.1060028076171875 seconds ---
Optimization Error:
4.502731778432756e-11


┌─────────────────────────┐┌────────────────────────┐»
q_0: ┤0                        ├┤0                       ├»
     │                         ││                        │»
q_1: ┤1 exp(-it ZXY)(-0.93556) ├┤1 exp(-it YXZ)(-1.1781) ├»
     │                         ││                        │»
q_2: ┤2                        ├┤2                       ├»
     └─────────────────────────┘└────────────────────────┘»
«     ┌───────────────────────┐┌──────────────────────┐»
«q_0: ┤0                      ├┤0                     ├»
«     │                       ││                      │»
«q_1: ┤1 exp(-it XYZ)(1.2793) ├┤1 exp(-it YZX)(1.189) ├»
«     │                       ││                      │»
«q_2: ┤2                      ├┤2                     ├»
«     └───────────────────────┘└──────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐ ░ »
«q_0: ┤0                       ├┤0                       ├─░─»
«     │                        ││                        │ ░ »
«q_1: ┤1 exp(-it XZY)(0.30774) ├┤1 exp(-it ZYX)(0.41544) ├─░─»
«     │                        ││                        │ ░ »
«q_2: ┤2                       ├┤2                       ├─░─»
«     └────────────────────────┘└────────────────────────┘ ░ »
«     ┌───────────────────┐┌───────────────────┐┌───────────────────┐ ░ »
«q_0: ┤0                  ├┤0                  ├┤0                  ├─░─»
«     │                   ││                   ││                   │ ░ »
«q_1: ┤1 exp(-it YIY)(-6) ├┤1 exp(-it IYY)(-3) ├┤1 exp(-it YYI)(-3) ├─░─»
«     │                   ││                   ││                   │ ░ »
«q_2: ┤2                  ├┤2                  ├┤2                  ├─░─»
«     └───────────────────┘└───────────────────┘└───────────────────┘ ░ »
«     ┌─────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                        ├┤0                        ├»
«     │                         ││                         │»
«q_1: ┤1 exp(-it ZYX)(-0.41544) ├┤1 exp(-it XZY)(-0.30774) ├»
«     │                         ││                         │»
«q_2: ┤2                        ├┤2                        ├»
«     └─────────────────────────┘└─────────────────────────┘»
«     ┌───────────────────────┐┌────────────────────────┐»
«q_0: ┤0                      ├┤0                       ├»
«     │                       ││                        │»
«q_1: ┤1 exp(-it YZX)(-1.189) ├┤1 exp(-it XYZ)(-1.2793) ├»
«     │                       ││                        │»
«q_2: ┤2                      ├┤2                       ├»
«     └───────────────────────┘└────────────────────────┘»
«     ┌───────────────────────┐┌────────────────────────┐
«q_0: ┤0                      ├┤0                       ├
«     │                       ││                        │
«q_1: ┤1 exp(-it YXZ)(1.1781) ├┤1 exp(-it ZXY)(0.93556) ├
«     │                       ││                        │
«q_2: ┤2                      ├┤2                       ├
«     └───────────────────────┘└────────────────────────┘